In [2]:
# ============================================================
# Imports & Config
# ============================================================
import numpy as np
import pandas as pd
import sys
from pathlib import Path
from typing import Dict, Tuple, Optional

ROOT_DIR = Path.cwd().parent
sys.path.append(str(ROOT_DIR))

from src.data_pipeline import oleo_farelo, oleo_palma, oleo_diesel, oil_share
from src.visualization import plot_ratio_std_plotly
from src.utils import apply_theme, date_range_picker, rsi, section
from src.email_utils import send_email_with_chart_attachments, EmailConfig


# ============================================================
# Configuration - MUDE AQUI
# ============================================================
CONFIG = {
    # Data and chart settings
    "start_date": "2025-01-01",
    "subplot_option": "RSI",  # ["Rolling STD", "RSI"]
    "rsi_length": 14,
    "ma_windows": [90],
    "rolling_window_std": 90,
    
    # Email settings
    "send_email": True,
    "email_recipients": [
    "diego.santanna@oleoplan.com.br",
    "otavio.kucharski@oleoplan.com.br",
    "daniel.roesch@oleoplan.com.br",
    "marcel.kuhn@oleoplan.com.br",
    "marcos.leite@oleoplan.com.br",
    "aldryn.comecanha@oleoplan.com.br",
    "leandro.ries@oleoplan.com.br",
    "matheus.castro@oleoplan.com.br",
    "diego.geittens@oleoplan.com.br",
    "guilherme.paese@oleoplan.com.br",
    "bruna.rosa@oleoplan.com.br",
    "chaiane.hochmuller@oleoplan.com.br",
    "catarina.souza@oleoplan.com.br",
    "gabriel.zanette@oleoplan.com.br",
    "rodrigo.lemanski@oleoplan.com.br",
    "fabio.magdaleno@oleoplan.com.br",
    ],
    "email_subject": "Market Monitor - Relatório de Ratios",
    "email_body": (
        "Segue abaixo o relatório atualizado com as análises das relações de "
        "preço de commodities considerando o último settlement price.\n\n"
        "As relações são construídas após conversão das unidades dos ativos para toneladas, "
        "sempre utilizando o continuation future 1"
    ),
    "email_footer": "Este email foi gerado automaticamente pelo Market Monitor Panel. https://market-monitor-op.streamlit.app/",
    "show_timestamp": False,  # Mostrar data/hora no email
}

# Ratio definitions: name -> (dataframe, column_name)
RATIOS: Dict[str, Tuple[pd.DataFrame, str]] = {
    "Óleo/Farelo": (oleo_farelo, "oleo_farelo"),
    "Óleo/Palma": (oleo_palma, "oleo_palma"),
    "Óleo/Diesel": (oleo_diesel, "oleo_diesel"),
    "Oil Share CME": (oil_share, "oil_share"),
}

# Select which ratios to process (add/remove as needed)
RATIOS_TO_PROCESS = ["Óleo/Farelo", "Óleo/Palma", "Óleo/Diesel"]


# ============================================================
# Helper Functions
# ============================================================
def prepare_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Convert date column to datetime."""
    df_copy = df.copy()
    df_copy["date"] = pd.to_datetime(df_copy["date"], errors="coerce")
    return df_copy


def get_last_valid_date(df: pd.DataFrame, exclude_today: bool = True) -> Optional[pd.Timestamp]:
    """Get the most recent date from dataframe, optionally excluding today."""
    if df["date"].isna().all():
        return None
    
    today = pd.to_datetime("today").normalize()
    
    if exclude_today:
        valid_dates = df[df["date"] < today]["date"]
        return valid_dates.max() if not valid_dates.empty else None
    
    return df["date"].max()


def filter_data(
    df: pd.DataFrame,
    column: str,
    start_date: pd.Timestamp,
    end_date: Optional[pd.Timestamp]
) -> pd.DataFrame:
    """Filter dataframe by date range and return cleaned view."""
    if end_date is None:
        return pd.DataFrame()
    
    mask = (df["date"] >= start_date) & (df["date"] <= end_date)
    return df.loc[mask, ["date", column]].dropna().sort_values("date")


def create_ratio_chart(
    data: pd.DataFrame,
    y_column: str,
    ratio_name: str,
    config: dict
) -> Optional[object]:
    """Create a ratio chart with configured parameters."""
    if data.empty:
        print(f"Sem dados em {ratio_name} no período selecionado.")
        return None
    
    subplot_key = "std" if config["subplot_option"] == "Rolling STD" else "rsi"
    
    fig = plot_ratio_std_plotly(
        x=data["date"],
        y=data[y_column],
        title=f"Relação {ratio_name}",
        ylabel=f"Relação {ratio_name}",
        rolling_window=config["rolling_window_std"],
        label_series=ratio_name,
        subplot=subplot_key,
        rsi_len=config["rsi_length"],
        rsi_fn=rsi,
        ma_windows=config["ma_windows"],
    )
    
    fig.update_layout(
        title=dict(pad=dict(b=12), x=0.0, xanchor="left", y=0.98, yanchor="top"),
        margin=dict(t=80),
    )
    
    fig.show()
    return fig


# ============================================================
# Main Execution
# ============================================================
def main():
    """Main execution function."""
    start_date = pd.to_datetime(CONFIG["start_date"])
    charts_to_send = {}
    
    for ratio_name in RATIOS_TO_PROCESS:
        if ratio_name not in RATIOS:
            print(f"⚠️  Ratio '{ratio_name}' não encontrado na configuração.")
            continue
        
        df, y_column = RATIOS[ratio_name]
        
        # Prepare data
        df_prepared = prepare_dataframe(df)
        last_date = get_last_valid_date(df_prepared)
        
        # Filter data
        view = filter_data(df_prepared, y_column, start_date, last_date)
        
        # Create chart
        fig = create_ratio_chart(view, y_column, ratio_name, CONFIG)
        
        if fig is not None:
            charts_to_send[ratio_name] = fig
    
    # Send email if configured
    if CONFIG["send_email"] and charts_to_send:
        print("\n" + "="*60)
        print("📧 Enviando email com os gráficos...")
        print("="*60)
        
        # Create email configuration from notebook CONFIG
        email_config = EmailConfig(
            recipients=CONFIG["email_recipients"],
            subject=CONFIG["email_subject"],
            body_text=CONFIG["email_body"],
            footer_text=CONFIG["email_footer"],
            show_timestamp=CONFIG.get("show_timestamp", True),
        )
        
        send_email_with_chart_attachments(
            charts=charts_to_send,
            config=email_config,
        )
        print("="*60)


# Execute
if __name__ == "__main__":
    main()
else:
    main()  # For notebook execution


📧 Enviando email com os gráficos...
✅ Gráfico 'Óleo/Farelo' convertido para imagem
✅ Gráfico 'Óleo/Palma' convertido para imagem
✅ Gráfico 'Óleo/Diesel' convertido para imagem

✅ Email enviado com sucesso para diego.santanna@oleoplan.com.br; otavio.kucharski@oleoplan.com.br; daniel.roesch@oleoplan.com.br; marcel.kuhn@oleoplan.com.br; marcos.leite@oleoplan.com.br; aldryn.comecanha@oleoplan.com.br; leandro.ries@oleoplan.com.br; matheus.castro@oleoplan.com.br; diego.geittens@oleoplan.com.br; guilherme.paese@oleoplan.com.br; bruna.rosa@oleoplan.com.br; chaiane.hochmuller@oleoplan.com.br; catarina.souza@oleoplan.com.br; gabriel.zanette@oleoplan.com.br; rodrigo.lemanski@oleoplan.com.br; fabio.magdaleno@oleoplan.com.br
   📊 3 gráfico(s) embutido(s) no corpo
🗑️  Arquivo temporário removido: Óleo/Farelo
🗑️  Arquivo temporário removido: Óleo/Palma
🗑️  Arquivo temporário removido: Óleo/Diesel
